In [2]:
import Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")

   Resolving package versions...
  No Changes to `C:\Users\javie\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\javie\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\javie\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\javie\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\javie\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\javie\.julia\environments\v1.10\Manifest.toml`


In [5]:
using JuMP, GLPK

# Definir los datos
oferta = [400, 200, 150]  # oferta de cada centro
demanda = [100, 200, 150, 160, 140]  # demanda de cada concesionario
distancias = [
    [100, 150, 200, 140, 35],  # Distancias desde centro 1
    [50, 70, 60, 65, 80],      # Distancias desde centro 2
    [40, 90, 100, 150, 130]    # Distancias desde centro 3
]

# Convertir a camiones
oferta = [ceil(value/18) for value in oferta] 
demanda = [ceil(value/18) for value in demanda]  

# Convertimos las distancias a costos multiplicándolas por 25 (costo por milla)
costo = [d .* 25 for d in distancias]

# Crear el modelo
model = Model(GLPK.Optimizer)

# Definimos las variables de decisión: cantidad enviada de cada centro a cada concesionario
@variable(model, x[1:3, 1:5] >= 0)  # x[i,j] es la cantidad enviada desde centro i a concesionario j

# Definimos la función objetivo: minimizar el costo total de transporte
@objective(model, Min, sum(x[i,j] * costo[i][j] for i in 1:3, j in 1:5))

# Restricciones de oferta: no se puede enviar más de lo disponible en cada centro
for i in 1:3
    @constraint(model, sum(x[i,j] for j in 1:5) <= oferta[i])
end

# Restricciones de demanda: cada concesionario debe recibir lo que necesita
for j in 1:5
    @constraint(model, sum(x[i,j] for i in 1:3) == demanda[j])
end

# Resolución del problema
optimize!(model)

println("Distribución óptima de automóviles:")
for i in 1:3
    for j in 1:5
        println("Desde centro $i a concesionario $j: ", value(x[i,j]))
    end
end

println("Costo total: ", objective_value(model))


Distribución óptima de automóviles:
Desde centro 1 a concesionario 1: 0.0
Desde centro 1 a concesionario 2: 6.0
Desde centro 1 a concesionario 3: 0.0
Desde centro 1 a concesionario 4: 9.0
Desde centro 1 a concesionario 5: 8.0
Desde centro 2 a concesionario 1: 0.0
Desde centro 2 a concesionario 2: 3.0
Desde centro 2 a concesionario 3: 9.0
Desde centro 2 a concesionario 4: 0.0
Desde centro 2 a concesionario 5: 0.0
Desde centro 3 a concesionario 1: 6.0
Desde centro 3 a concesionario 2: 3.0
Desde centro 3 a concesionario 3: 0.0
Desde centro 3 a concesionario 4: 0.0
Desde centro 3 a concesionario 5: 0.0
Costo total: 92500.0
